# Parse the HTML files , convert to DataFrames , clean the data

#### To learn more about selecting from dropdown with selenium : https://www.youtube.com/watch?v=YWGyLGuY67g
#### To learn more about web scraping : https://www.youtube.com/watch?v=JGQGd-oa0l4&t=57s
### I use BeatifulSoup to parse the html 



In [1]:
from bs4 import BeautifulSoup
import pandas as pd

pd.set_option("display.max_columns", None)
%load_ext lab_black

In [2]:
season_types = ["Regular%20Season", "Playoffs"]

In [3]:
# to get the list of seasons
y1 = [str(item) for item in list(range(1996, 2022))]
y2 = [str(item)[-2:] for item in list(range(1997, 2023))]
seasons = [y1[i] + "-" + y2[i] for i in range(len(y1))]
seasons

['1996-97',
 '1997-98',
 '1998-99',
 '1999-00',
 '2000-01',
 '2001-02',
 '2002-03',
 '2003-04',
 '2004-05',
 '2005-06',
 '2006-07',
 '2007-08',
 '2008-09',
 '2009-10',
 '2010-11',
 '2011-12',
 '2012-13',
 '2013-14',
 '2014-15',
 '2015-16',
 '2016-17',
 '2017-18',
 '2018-19',
 '2019-20',
 '2020-21',
 '2021-22']

In [4]:
# to open and parse the saved html
with open("nba_stats.html") as f:
    page = f.read()


dfs = list()
for season in seasons:
    for season_type in season_types:
        with open(f"""nba_stats_{season}_{season_type.split("%")[0]}.html""") as f:
            page = f.read()
            soup = BeautifulSoup(page, "html.parser")
            html_table = soup.find("div", class_="nba-stat-table")
            df = pd.read_html(str(html_table))[0]
            df["season"] = (
                season.split("-")[0][:2] + season.split("-")[-1][-2:]
            )  # the season is defined as ending year (watch out season 2000)
            df["tournement"] = season_type.split("%20")[-1]
            dfs.append(df)

In [5]:
# concatinate all the dataframes
raw = pd.concat(dfs).reset_index(drop=True)
df = raw.copy()
df.drop(columns="Unnamed: 0", inplace=True)

In [6]:
# capitalize all the column names
df.columns = [item.capitalize() for item in df.columns]

In [7]:
# drop columns with no values
blank_cols = df.columns[df.isna().all()].tolist()
df.drop(columns=blank_cols, inplace=True)

In [8]:
df.Season.unique()

array(['1997', '1998', '1999', '1900', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022'], dtype=object)

In [9]:
# Season 1900 is a mistake resulting from the algoritm in the html file read loop
df.Season = df.Season.apply(lambda year: "2000" if year == "1900" else year)
df.Season = df.Season.astype(int)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17526 entries, 0 to 17525
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player      17526 non-null  object 
 1   Team        17526 non-null  object 
 2   Age         17526 non-null  int64  
 3   Gp          17526 non-null  int64  
 4   W           17526 non-null  int64  
 5   L           17526 non-null  int64  
 6   Min         17526 non-null  int64  
 7   Pts         17526 non-null  int64  
 8   Fgm         17526 non-null  int64  
 9   Fga         17526 non-null  int64  
 10  Fg%         17526 non-null  float64
 11  3pm         17526 non-null  int64  
 12  3pa         17526 non-null  int64  
 13  3p%         17526 non-null  float64
 14  Ftm         17526 non-null  int64  
 15  Fta         17526 non-null  int64  
 16  Ft%         17526 non-null  float64
 17  Oreb        17526 non-null  int64  
 18  Dreb        17526 non-null  int64  
 19  Reb         17526 non-nul

In [11]:
df

,Player,Team,Age,Gp,W,L,Min,Pts,Fgm,Fga,Fg%,3pm,3pa,3p%,Ftm,Fta,Ft%,Oreb,Dreb,Reb,Ast,Tov,Stl,Blk,Pf,Fp,Dd2,Td3,+/-,Season,Tournement
0,A.C. Green,DAL,33,83,23,60,2494,597,234,484,48.3,1,20,5.0,128,197,65.0,222,434,656,69,74,70,16,145,1671.7,16,0,-350,1997,Season
1,Aaron McKie,DET,24,83,48,35,1624,433,150,365,41.1,41,103,39.8,92,110,83.6,40,181,221,161,90,77,22,130,1146.7,1,0,133,1997,Season
2,Michael Finley,DAL,24,83,23,60,2789,1249,475,1071,44.4,101,280,36.1,198,245,80.8,88,284,372,224,164,68,24,138,2143.4,1,0,-315,1997,Season
3,Antoine Carr,UTA,35,82,64,18,1463,603,252,522,48.3,0,3,0.0,99,127,78.0,60,135,195,74,75,24,63,214,1134.0,0,0,90,1997,Season
4,Antoine Walker,BOS,20,82,15,67,2972,1437,577,1355,42.6,52,159,32.7,231,366,63.1,288,453,741,262,230,105,53,271,2963.2,33,2,-556,1997,Season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17521,Josh Okogie,MIN,23,1,0,1,2,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,2,2022,Playoffs
17522,Markieff Morris,MIA,32,1,0,1,3,0,0,1,0.0,0,0,0.0,0,0,0.0,0,1,1,0,1,0,0,2,0.2,0,0,2,2022,Playoffs
17523,Nickeil Alexander-Walker,UTA,23,1,0,1,5,5,2,2,100.0,0,0,0.0,1,1,100.0,0,1,1,1,0,1,0,0,10.7,0,0,4,2022,Playoffs
17524,Paul Millsap,PHI,37,1,0,1,6,0,0,0,0.0,0,0,0.0,0,0,0.0,0,1,1,1,0,0,0,1,2.7,0,0,-2,2022,Playoffs


In [12]:
df.Team.unique()

array(['DAL', 'DET', 'UTA', 'BOS', 'IND', 'LAC', 'CLE', 'WAS', 'ATL',
       'PHI', 'DEN', 'ORL', 'MIL', 'SEA', 'POR', 'SAS', 'MIA', 'NJN',
       'HOU', 'CHI', 'SAC', 'VAN', 'MIN', 'NYK', 'TOR', 'CHH', 'LAL',
       'GSW', 'PHX', 'MEM', 'NOH', 'CHA', 'NOK', 'OKC', 'BKN', 'NOP'],
      dtype=object)

In [13]:
# These are same city teams:
def Team_consolidate(team):
    if team == "NJN":
        return "BKN"
    if team == "CHH":
        return "CHA"
    if team in ["NOK", "NOH"]:
        return "NOP"
    else:
        return team


df.Team = df.Team.apply(Team_consolidate)

In [14]:
# file with team location info
df_locations = pd.read_csv("team_locations.csv")

# file with team code and team names
df_codes = pd.read_csv("city_codes.csv")

In [15]:
df_locations.columns, df_codes.columns

(Index(['Team Name', 'Arena Name', 'Arena Location', 'Seating Capacity',
        'Opening Year'],
       dtype='object'),
 Index(['Unnamed: 0', 'code', 'city'], dtype='object'))

In [16]:
df_team_loc = pd.merge(
    df_locations,
    df_codes,
    how="left",
    left_on="Team Name",
    right_on="city",
    indicator=True,
)


df_team_loc.code = df_team_loc.code.apply(lambda name: "WAS" if name == "WSH" else name)

df_team_loc

,Team Name,Arena Name,Arena Location,Seating Capacity,Opening Year,Unnamed: 0,code,city,_merge
0,Atlanta Hawks,State Farm Arena,"Atlanta, Georgia",18118,1999,1,ATL,Atlanta Hawks,both
1,Boston Celtics,TD Garden,"Boston, Massachusetts",18624,1995,3,BOS,Boston Celtics,both
2,Brooklyn Nets,Barclays Center,"Brooklyn, New York",17732,2012,2,BKN,Brooklyn Nets,both
3,Charlotte Hornets,Spectrum Center,"Charlotte, North Carolina",19077,2005,4,CHA,Charlotte Hornets,both
4,Chicago Bulls,United Center,"Chicago, Illinois",20917,1994,5,CHI,Chicago Bulls,both
5,Cleveland Cavaliers,Rocket Mortgage Fieldhouse,"Cleveland, Ohio",19432,1994,6,CLE,Cleveland Cavaliers,both
6,Dallas Mavericks,American Airlines Center,"Dallas, Texas",19200,2001,7,DAL,Dallas Mavericks,both
7,Denver Nuggets,Pepsi Center,"Denver, Colorado",19520,1999,8,DEN,Denver Nuggets,both
8,Detroit Pistons,Little Caesars Arena,"Detroit, Michigan",20491,2017,9,DET,Detroit Pistons,both
9,Golden State Warriors,Chase Center,"San Francisco, California",18064,2019,10,GSW,Golden State Warriors,both


In [17]:
# drop unnecessary columns
df_team_loc.drop(
    ["Arena Name", "Seating Capacity", "Opening Year", "Unnamed: 0", "city", "_merge"],
    axis=1,
    inplace=True,
)

In [18]:
df_team_loc

,Team Name,Arena Location,code
0,Atlanta Hawks,"Atlanta, Georgia",ATL
1,Boston Celtics,"Boston, Massachusetts",BOS
2,Brooklyn Nets,"Brooklyn, New York",BKN
3,Charlotte Hornets,"Charlotte, North Carolina",CHA
4,Chicago Bulls,"Chicago, Illinois",CHI
5,Cleveland Cavaliers,"Cleveland, Ohio",CLE
6,Dallas Mavericks,"Dallas, Texas",DAL
7,Denver Nuggets,"Denver, Colorado",DEN
8,Detroit Pistons,"Detroit, Michigan",DET
9,Golden State Warriors,"San Francisco, California",GSW


In [19]:
# insert data for 2 missing teams
df_team_loc = pd.concat(
    [
        df_team_loc,
        pd.DataFrame(
            [
                ["Seattle SuperSonics", "Seattle, Washington", "SEA"],
                ["Vancouver Grizzlies", "vancouver, British Columbia", "VAN"],
            ],
            columns=df_team_loc.columns.to_list(),
        ),
    ],
    ignore_index=True,
)

df_team_loc

,Team Name,Arena Location,code
0,Atlanta Hawks,"Atlanta, Georgia",ATL
1,Boston Celtics,"Boston, Massachusetts",BOS
2,Brooklyn Nets,"Brooklyn, New York",BKN
3,Charlotte Hornets,"Charlotte, North Carolina",CHA
4,Chicago Bulls,"Chicago, Illinois",CHI
5,Cleveland Cavaliers,"Cleveland, Ohio",CLE
6,Dallas Mavericks,"Dallas, Texas",DAL
7,Denver Nuggets,"Denver, Colorado",DEN
8,Detroit Pistons,"Detroit, Michigan",DET
9,Golden State Warriors,"San Francisco, California",GSW


In [20]:
df = pd.merge(
    df, df_team_loc, how="left", left_on="Team", right_on="code", indicator=True
)

In [21]:
# to check there is no missing merge
df[df._merge == "left_only"]

,Player,Team,Age,Gp,W,L,Min,Pts,Fgm,Fga,Fg%,3pm,3pa,3p%,Ftm,Fta,Ft%,Oreb,Dreb,Reb,Ast,Tov,Stl,Blk,Pf,Fp,Dd2,Td3,+/-,Season,Tournement,Team Name,Arena Location,code,_merge


In [22]:
# _merge columns is not necessary anymore
df.drop("_merge", axis=1, inplace=True)

In [23]:
# capitalize all the column names
df.columns = [item.capitalize() for item in df.columns]
df.drop("Code", axis=1, inplace=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17526 entries, 0 to 17525
Data columns (total 33 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          17526 non-null  object 
 1   Team            17526 non-null  object 
 2   Age             17526 non-null  int64  
 3   Gp              17526 non-null  int64  
 4   W               17526 non-null  int64  
 5   L               17526 non-null  int64  
 6   Min             17526 non-null  int64  
 7   Pts             17526 non-null  int64  
 8   Fgm             17526 non-null  int64  
 9   Fga             17526 non-null  int64  
 10  Fg%             17526 non-null  float64
 11  3pm             17526 non-null  int64  
 12  3pa             17526 non-null  int64  
 13  3p%             17526 non-null  float64
 14  Ftm             17526 non-null  int64  
 15  Fta             17526 non-null  int64  
 16  Ft%             17526 non-null  float64
 17  Oreb            17526 non-null 

In [25]:
df[df.Player == "Stephen Curry"]

,Player,Team,Age,Gp,W,L,Min,Pts,Fgm,Fga,Fg%,3pm,3pa,3p%,Ftm,Fta,Ft%,Oreb,Dreb,Reb,Ast,Tov,Stl,Blk,Pf,Fp,Dd2,Td3,+/-,Season,Tournement,Team name,Arena location
8339,Stephen Curry,GSW,22,80,25,55,2896,1399,528,1143,46.2,166,380,43.7,177,200,88.5,48,308,356,472,243,152,19,252,2804.2,12,1,-187,2010,Season,Golden State Warriors,"San Francisco, California"
9038,Stephen Curry,GSW,23,74,33,41,2489,1373,505,1053,48.0,151,342,44.2,212,227,93.4,52,234,286,432,226,109,20,233,2525.2,5,0,-17,2011,Season,Golden State Warriors,"San Francisco, California"
9926,Stephen Curry,GSW,24,26,13,13,732,383,145,296,49.0,55,121,45.5,38,47,80.9,15,73,88,138,65,39,8,62,771.6,3,0,79,2012,Season,Golden State Warriors,"San Francisco, California"
10333,Stephen Curry,GSW,25,78,45,33,2983,1786,626,1388,45.1,272,600,45.3,262,291,90.0,59,255,314,539,240,126,12,198,3145.3,15,0,133,2013,Season,Golden State Warriors,"San Francisco, California"
10759,Stephen Curry,GSW,25,12,6,6,497,281,102,235,43.4,42,106,39.6,35,38,92.1,5,41,46,97,40,20,2,35,507.7,3,0,52,2013,Playoffs,Golden State Warriors,"San Francisco, California"
11008,Stephen Curry,GSW,26,78,50,28,2846,1873,652,1383,47.1,261,615,42.4,308,348,88.5,46,288,334,666,294,128,14,194,3404.8,28,4,574,2014,Season,Golden State Warriors,"San Francisco, California"
11517,Stephen Curry,GSW,26,7,3,4,296,161,51,116,44.0,22,57,38.6,37,42,88.1,4,21,25,59,26,12,1,18,292.5,1,0,3,2014,Playoffs,Golden State Warriors,"San Francisco, California"
11647,Stephen Curry,GSW,27,80,67,13,2613,1900,653,1341,48.7,286,646,44.3,308,337,91.4,56,285,341,619,249,163,16,158,3525.7,23,0,919,2015,Season,Golden State Warriors,"San Francisco, California"
12093,Stephen Curry,GSW,27,21,16,5,826,594,200,439,45.6,98,232,42.2,96,115,83.5,20,85,105,134,82,39,3,47,965.0,1,0,166,2015,Playoffs,Golden State Warriors,"San Francisco, California"
12373,Stephen Curry,GSW,28,79,71,8,2700,2375,805,1598,50.4,402,886,45.4,363,400,90.8,68,362,430,527,262,169,15,161,3971.5,15,2,1022,2016,Season,Golden State Warriors,"San Francisco, California"


In [26]:
df.to_csv("nba_player_stats.csv")

+ I created one single dataframe for all seasons 
+ I added geographoical team information to get some map view later 
